# Setting up and Facilitating SAMOO Runs
### Walkthrough of SAMOO Utility

Recall that Surrogate-Assisted Multi-Objective Optimisation (SAMOO) follows this general algorithm:
1. LHS sampling of initial training dataset and starting population
2. Complex model evaluation (hereinafter referred to as Outer Iteration)
3. Pareto dominance evaluation
     - if front converged: exit; else: continue to step 4
4. (Re)training the GPR
5. MOO with GPR replacing the complex model (hereinafter referred to as Inner Iterations)
6. Resampling for new training points (also called infills)

This entire process requires two models to be set up: a process-based model (PBM), which is a complex, expensive-to-evaluate model such as a groundwater numerical model, and a surrogate model, which is a fast-to-evaluate model, such as a GPR, that emulates the response of the PBM at some specified input parameters (i.e., decision vaiable values). PBM is run in Step 2 of the algorithm while the surrogate model is run in Step 5.

pestpp-mou can be used to run both the outer and inner iterations of the SAMOO algorithm, where in the outer iteration it simply runs the PBM and records the results, and in the inner iteration it runs the MOO with the GPR surrogate model. To do this, we need to set up pestpp-mou control files for the outer and inner iterations. We have shown in the previous tutorial a glimpse of how this can be facilitated using the samoo utility. Here, we will show a more detailed walkthrough on how to set up the SAMOO runs using the utility.

We need three sets of control files, along with input and template files required by pestpp-mou. For better housekeeping, we will place each set of files in separate template directories:
1. template_outer -- for the outer iteration
2. template_inner -- for the inner iteration
3. template_repo_update -- for updating the outer iteration repository

These will be generated by the samoo utility with the help of pyemu tools. Let us first load the files we need and the samoo utility.



In [4]:
import os
import sys
import shutil
import pandas as pd
import numpy as np

base_d = "./demo_files/kursawe2d"
assert os.path.exists(base_d)

temp_d = "samoo_util_demo"

sys.path.insert(0,temp_d)
from bin.samoo_utils import SAMOO
optimizer = SAMOO()

At this point, we only need the over-all template directory containing all the standard files needed by pestpp-mou. These files have been prepared for you in the `samoo_util_demo/template` directory (run the cell below). We will still use the 2D Kursawe problem.

In [5]:
if os.path.exists(temp_d):
    shutil.rmtree(temp_d)
shutil.copytree(os.path.join(base_d, 'kursawe2d_pbm_template'), os.path.join(temp_d, 'template'))

print("Files in template directory:")
for f in os.listdir(os.path.join(temp_d, 'template')):
    print(f"  {f}")

Files in template directory:
  dv_tpl.dat
  forward_gprun.py
  forward_pbrun.py
  kur.pst
  output.ins


Feel free to explore the contents of these files.
* `dv_tpl.dat` - this is the template file for the decision variables
* `output.ins` - this contains the instructions on how to read the output from the model. Notice that included here are the outputs related to standard deviation of the surrogate model predictions. Although this is irrelevant for outer iterations, we still keep it here in the general templateso that it can be passed to the inner iteration when we generate the relevant files later.
* `kur.pst` - this is the pest control file. You can generate this on your own using your own scripts. Generate this as if you are setting up a conventional pestpp-mou run that only operates the PBM.
* `forward_pbrun.py` - this is the script that runs the PBM. Set the standard deviation outputs to zero.
* `forward_gprun.py` - this is the script that runs the GPR.







In [6]:
optimizer.nmax_inner = 20 #the number of inner iterations
optimizer.ppd_beta = 0.6 #discussed in Part 2

swarm_size = 40
bounds = np.array([[-5, 5] for i in range(2)])
from bin.LHS_sampler import generate_lhstrainingset
generate_lhstrainingset(os.path.join(temp_d, 'template'), seed=42, n_samples=swarm_size, n_dimensions=2, bounds=bounds)

os.chdir('samoo_util_demo')


In [11]:
optimizer.prep_templates()


2025-05-23 14:52:27.778176: prepping templates 


2025-05-23 14:52:27.788635: prepping outer template 

noptmax:-1, npar_adj:2, nnz_obs:4

2025-05-23 14:52:28.024765: outer template prepped 


2025-05-23 14:52:28.024765: prepping repo update template 

noptmax:-1, npar_adj:2, nnz_obs:4

2025-05-23 14:52:28.189379: outer repo update template prepped 


2025-05-23 14:52:28.189379: prepping inner template 

noptmax:20, npar_adj:2, nnz_obs:4

2025-05-23 14:57:04.884366: inner template prepped 

